# Airline staffing problem

*Original lab session by Gérard Verfaillie.*

This problem is about the staffing of an airline, i.e. assigning pilots and crew members to scheduled flights using a Constraint Satisfaction/Optimisation Problem (CSP/COP) solver (here `facile`). Three instances of the problem are given.

As a pure CSP modelling fails for the larger instance, we will solve it using a combination of CSP and Mixed-Integer Linear Programming (MILP) modelling. You may refer to past classes with the `PuLP` library for basic usage.

<div class="alert alert-warning">
    <b>Warning</b>: You may submit in French or English. Make a choice and stick to it (don't mix languages).
</div>

In [1]:
import facile

## Reminders of syntax with the facile library

<div class="alert alert-success">
    <b>Hint</b>: Constraints may be summed: the result is the number of True values in the set of constraints.<br/>(However, this doesn't work with global constraints such as alldifferent)
</div>

Example: Find an array of length $n$ taking values in $[0, n-1]$ so that the $i$-th value is equal to the number of $i$ inside the array.

In [ ]:
n = 10
array = [facile.variable(range(n)) for i in range(n)]

for i in range(n):
    facile.constraint(sum([x == i for x in array]) == array[i])

if facile.solve(array):
    print(f"indices: {list(range(n))}")
    print(f"array:   {[v.value() for v in array]}")

<div class="alert alert-success">
    <b>Hint</b>: You may use the & and | operators for the logical $\land$ and $\lor$.<br/>(However, this doesn't work with global constraints such as alldifferent)
</div>

In [ ]:
a, b = facile.variable([0, 1, 2]), facile.variable([0, 1, 2])

left = (a == b) & (a + b == 2)
right = a > b
facile.constraint(left | right)
result = facile.solve([a, b])
a.value(), b.value()

<div class="alert alert-success">
    <b>Hint</b>: You may use the <code>solve_all</code> function to get all the solutions to a CSP problem.
</div>

In [ ]:
a, b = facile.variable([0, 1, 2]), facile.variable([0, 1, 2])

left = (a == b) & (a + b == 2)
right = a > b
facile.constraint(left | right)
result = facile.solve_all([a, b])

[x.solution for x in result]

<div class="alert alert-warning">
    <b>Warning</b>: Using <code>solve_all</code> does not assign any value to a variable. (The explanation lies in the fact the Branch&Bound algorithm returns with no solution found—hence the last <code>None</code>)<br/>Note the similar behaviour with the <code>minimize</code> function.
</div>

In [ ]:
a.value(), b.value()

<div class="alert alert-success">
    <b>Hint</b>: You may index a 1D-array of variables with a facile variable or expression. But you first need to wrap the array with the facile.array function.
</div>
<div class="alert alert-danger">
    <b>Warning</b>: Be aware that the indexing value will be constrained between 0 and max_index.
</div>
<div class="alert alert-warning">
    <b>Warning</b>: If you work with 2D-arrays, the linearisation of indices (remember your programming classes 😀) is left to you as part of the exercice. This should be something along a[i][j] becomes a[i*n + j]...
</div>

Example: find the position of the first 0 in a given array.

In [ ]:
a = facile.array([1, 2, 0, 4, 5, 0])
i = facile.variable(range(len(a)))
facile.constraint(a[i] == 0)
result = facile.minimize([i], i)
result.solution

## Problem description

An airline operates a given set of flights between pairs of airports. Each flight must be staffed with two pilots and enough cabin crew members depending on the type of aircraft.

Each staff member must start its trip from his hometown (not before 12am) and be back home in the evening (before 12am). They must not fly more than a maximum number of flights per day, and more than a maximum amount of time out of home (from first take-off time to last landing time in addition to commuting time).

Each staff member may fly as a passenger if they just need to be transferred to another airport/city. They may as well be assigned to just stay home.

We will first search for a solution to this problem, then find the optimal solution that will maximise the money made by the airline by minimising the number of passenger seats assigned to staff.

## Problem data

The following data are given:
- `Nv`: total number of flights;
- `Na`: total number of airports;
- `Ni`: total number of cities;
- `Va`: a table associating a city to an airport (they may be several airports in the same city);
- `Ov` (resp. `Dv`): this table associates an origin (resp. destination) airport to a flight;
- `Td` (resp. `Ta`): this table associates a take-off time (resp. landing time) to a flight, (in round hours);
- `Dt`: this table returns transit times between two airports of the same city (in hours). This time is equal to 25 (more than a day, so infinite) for airports in different cities; `Dt[i][i]` is the time required to transfer between two aircraft at airport $i$;
- `Np`: this table associates a number of passenger seats to a flight;
- `Nec`: this table associates a number of required cabin crew members to a flight;
- `Pr`: this table associates a price for a ticket in a flight;
- `Ne`: total number of staff in the airline;
- `Ty`: this table associates 1 to a pilot, 0 to a cabin crew member;
- `Vh`: this tables associates a hometown to each staff member;
- `Nvmax`: maximum number of flights per staff member per day;
- `Dmax`: maximum work time (out of home) per staff member per day;
- `Dda`: standard commuting time (both ways).

<div class="alert alert-success">
    <b>Hint</b>: Three instances are given (tiny, small, normal). Start debugging with the tiny instance; when it works, try the small instance: <b>you will be graded based on the results of the small instance</b>.
</div>
<div class="alert alert-warning">
    <b>Warning</b>: Keep the normal instance for next section. (Trying it with this modelling may stall the kernel and require a restart)
</div>

Index 0 in the tables printed below means "no flight", associated to "no airport" and "no city".

In [ ]:
%run airline-tiny.py

In [2]:
%run airline-small.py

In [ ]:
%run airline-normal.py

## CSP modelling

### Variable definitions

First, define the variables of the problem. You may associate a variable to each flight taken by a staff member on a given day.

In [3]:
schedule = [[facile.variable(0,Nv) for j in range(Nvmax+1)] for i in range(Ne)]

### Constraints definition (50%)

<div class="alert alert-warning">
    <b>Warning</b>: Please write and explain (use Python comments) the code for each constraint in the corresponding <b>code</b> cells.<br/>Use Python comments for explanations. <b>You will be graded accordingly.</b><br/>You may add as many cells as necessary but do not move existing cells.
</div>

- Flights assigned to `0` (no flight) must be at the end of the schedule.  
  `[1, 0, 2]` could be equivalent to `[1, 2, 0]` but only the second one is valid.

In [4]:
for line in schedule:
    facile.constraint(line[0] == 0)
    for j in range(1,Nvmax):
        facile.constraint( (line[j] != 0) | (line[j+1] == 0) ) 

- If a staff member flies on that day, then (s)he leaves home and returns home.

In [5]:
Ov = facile.array(Ov)
Dv = facile.array(Dv)
Va = facile.array(Va)
Ta = facile.array(Ta)
Td = facile.array(Td)

lista = []
for line in Dt:
    for j in line:
        lista.append(j)
Dt1 = facile.array(lista)

for i in range(Ne):
    first_flight = schedule[i][1]
    hometown = Vh[i]
    citydeparture = Va[Ov[first_flight]]
    
    facile.constraint((citydeparture == hometown) | (first_flight == 0))
    
    staff = schedule[i][:]
    
    last_flight = sum([staff[i] != 0 for i in range(Nvmax+1)])
    staff = facile.array(staff)
    
    cityarrival = Va[Dv[staff[last_flight]]]
    
    facile.constraint((cityarrival == hometown) | (first_flight == 0))   

- A staff member may chain two flights iff :
    - first flight's destination airport and second flight's origin airport cities match;
    - there is enough transfer time between flights.

In [6]:
for i in range(Ne):
    for j in range(1,Nvmax):
        facile.constraint( (Va[Dv[schedule[i][j]]] == Va[Ov[schedule[i][j+1]]] )
                          |
                          (schedule[i][j+1] == 0) )
        facile.constraint( (Td[schedule[i][j+1]] - Ta[schedule[i][j]]
                          >=
                          Dt1[ Ov[schedule[i][j+1]] + (Na+1)*Dv[schedule[i][j]] ])
                         |
                         (schedule[i][j+1] == 0) )

- A staff member may not exceed his/her total time out of home;

In [7]:
for i in range(Ne):
    staff = facile.array(schedule[i])
    last_flight = sum(schedule[i][j]!=0 for j in range(Nvmax+1))
    total_time = Ta[staff[last_flight]] - Td[staff[1]] 
    facile.constraint(total_time <= Dmax - Dda)

- Each flights requires at least two pilots and enough cabin crew members.

In [8]:
empty = []
for line in schedule:
    for j in line:
        empty.append(j)
schedule1 = facile.array(empty)

for k in range(1,Nv+1):
    crew = sum([schedule1[i*(Nvmax+1)+j] == k for i in range(Ne) for j in range(1,Nvmax+1) if Ty[i] == 0])
    pilot = sum([schedule1[i*(Nvmax+1)+j] == k for i in range(Ne) for j in range(1,Nvmax+1) if Ty[i] == 1])
    facile.constraint(crew >= Nec[k])
    facile.constraint(pilot >= 2)

In [ ]:
facile.solve(sum(schedule,[]))

### Problem optimisation (5%)

In [11]:
revenue = facile.variable(0, 10000000)
for flight in range(1, Nv+1):
    crew = sum([schedule1[i*(Nvmax+1)+j] == flight for i in range(Ne) for j in range(1,Nvmax+1) if Ty[i] == 0])
    pilot = sum([schedule1[i*(Nvmax+1)+j] == flight for i in range(Ne) for j in range(1,Nvmax+1) if Ty[i] == 1])
    passagers = Np[flight] - (pilot - 2 + crew - Nec[flight])
    revenue = revenue + passagers*Pr[flight]

### Answer the problem (10%)

<div class='alert alert-warning'>
To maximise your grade, please post-process and pretty-print the results of the optimisation and check they are consistent with the constraints. <b>If you find inconsistencies you cannot solve, please point them.</b> (You will fail this question if you pretend everything is fine or if you try to hide your issues)
</div>

For each staff member, list the flights (s)he will be flying with cities and take-off/landing times.

Valid example:
```
Staff number 0: 1 [1, 2] 2 [2, 1] scheduled at [8, 9] [12, 13]
Staff number 1: ...
```
    
For each flight, list the pilots and cabin crew members flying (together with the required staffing of the aircraft).

Valid example:
```
Flight number 1: pilots {0, 1} cabin crew {2, 3} (≥ 2) ;
Flight number 2: ...
```


In [12]:
result = facile.minimize(schedule1, -revenue)
print(result)
print(result.solution)

Current evaluation            : -10079400
Current solution              : [0, 1, 4, 6, 0, 1, ...]
Resolution status             : True
Resolution time               : 1.4e+02s

[0, 1, 4, 6, 0, 1, 4, 6, 0, 2, 5, 0, 0, 2, 5, 0, 0, 3, 7, 5, 0, 3, 7, 5, 0, 1, 4, 6, 0, 1, 4, 6, 0, 2, 5, 0, 0, 2, 5, 0, 0, 3, 6, 0, 0, 3, 7, 5]


## A different modelling for CSP and MILP

Since bigger instances will require too much computing time, let's try something smarter:
1. First use a CSP solver to compute all possible paths between cities;
2. Then compare performances between a CSP and a MILP solver as we no longer assign staff members but groups of staff members to each path.

<div class="alert alert-success">
    <b>Hint</b>: Three instances are given (tiny, small, normal). Start debugging with the small instance; when it works, try the normal instance: you will be graded based on the results of the <b>normal instance</b>.
</div>

### First questions (10%)

<div class="alert alert-warning">
    <b>Warning</b>: Do not expect any external assistance for these questions. You are on your own.
</div>

1. Why would it help to solve the problem for groups of staff member rather than individuals. Which mathematical principle enters into account?
2. Try to explain why CSP better suits (compared to MILP) the path computing part of the problem.

### Compute all possible paths (10%)

In [ ]:
%run airline-small.py

In [ ]:
%run airline-normal.py

### CSP and MILP resolution (15%)

- Assign a group of staff members to each path;
- Compare the computing time with CSP and MILP resolutions (use `PuLP` library for MILP);
- Post-process and pretty-print the results;
- Comment